# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_sentences = source_text.split('\n')
    target_sentences = target_text.split('\n')
    
    source_id_text = []
    target_id_text = []
    
    for sentence in source_sentences:
        acc = []
        for word in sentence.split():
            acc.append(source_vocab_to_int[word])

        source_id_text.append(acc)
        
    for sentence in target_sentences:
        acc = []
        for word in sentence.split():
            acc.append(target_vocab_to_int[word])
        
        acc.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(acc)

    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    input_ = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, None, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, None, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, [None], 'target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, [None], 'source_sequence_length')
    
    return input_, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    go_ids = tf.constant([[target_vocab_to_int['<GO>']]]*batch_size)
    lasts = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    return tf.concat([go_ids, lasts], 1)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    def lstm_cell():
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        return tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)

    embedding = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    stacked = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
    return tf.nn.dynamic_rnn(stacked, embedding, source_sequence_length, dtype=tf.float32)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    help_ = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=help_, initial_state=encoder_state, output_layer=output_layer)
    logits, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, impute_finished=True, maximum_iterations=max_summary_length)
    return logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [7]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_ = tf.constant([start_of_sequence_id]*batch_size)
    help_ = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_, end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, help_, encoder_state, output_layer)
    logits, _ = tf.contrib.seq2seq.dynamic_decode(decoder, impute_finished=True, maximum_iterations=max_target_sequence_length)
    return logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [8]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    def lstm_cell():
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        return tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
    
    cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    embeddings = tf.Variable(tf.random_normal([target_vocab_size, decoding_embedding_size]))
    embed_input = tf.nn.embedding_lookup(embeddings, dec_input)

    with tf.variable_scope('decode_layer'):
        train_logits = decoding_layer_train(encoder_state, cells, embed_input, target_sequence_length, 
                                            max_target_sequence_length, output_layer, keep_prob)
    
    with tf.variable_scope('decode_layer', reuse=True):
        inference_logits = decoding_layer_infer(encoder_state, cells, embeddings, target_vocab_to_int['<GO>'], 
                                                target_vocab_to_int['<EOS>'], max_target_sequence_length, target_vocab_size, 
                                                output_layer, batch_size, keep_prob)
    
    return train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [9]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    encoded_input, encoded_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                                                  source_sequence_length, source_vocab_size, enc_embedding_size)
    decoded_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    return decoding_layer(decoded_input, encoded_state, target_sequence_length, max_target_sentence_length, 
                          rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, 
                          keep_prob, dec_embedding_size)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [18]:
# Number of Epochs
epochs = 12
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 227
decoding_embedding_size = 227
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.9
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2330, Validation Accuracy: 0.3099, Loss: 5.3156
Epoch   0 Batch    2/269 - Train Accuracy: 0.3007, Validation Accuracy: 0.3421, Loss: 4.5163
Epoch   0 Batch    3/269 - Train Accuracy: 0.2829, Validation Accuracy: 0.3442, Loss: 4.2938
Epoch   0 Batch    4/269 - Train Accuracy: 0.2692, Validation Accuracy: 0.3424, Loss: 4.0364
Epoch   0 Batch    5/269 - Train Accuracy: 0.2751, Validation Accuracy: 0.3525, Loss: 3.5661
Epoch   0 Batch    6/269 - Train Accuracy: 0.3631, Validation Accuracy: 0.3888, Loss: 3.2367
Epoch   0 Batch    7/269 - Train Accuracy: 0.3332, Validation Accuracy: 0.3612, Loss: 3.0984
Epoch   0 Batch    8/269 - Train Accuracy: 0.3168, Validation Accuracy: 0.3808, Loss: 3.0906
Epoch   0 Batch    9/269 - Train Accuracy: 0.3625, Validation Accuracy: 0.4026, Loss: 2.8987
Epoch   0 Batch   10/269 - Train Accuracy: 0.3515, Validation Accuracy: 0.4123, Loss: 2.8704
Epoch   0 Batch   11/269 - Train Accuracy: 0.3748, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4278, Validation Accuracy: 0.4959, Loss: 1.0398
Epoch   0 Batch   91/269 - Train Accuracy: 0.4485, Validation Accuracy: 0.4790, Loss: 0.9525
Epoch   0 Batch   92/269 - Train Accuracy: 0.4661, Validation Accuracy: 0.4794, Loss: 0.9626
Epoch   0 Batch   93/269 - Train Accuracy: 0.4919, Validation Accuracy: 0.4880, Loss: 0.9124
Epoch   0 Batch   94/269 - Train Accuracy: 0.4836, Validation Accuracy: 0.4922, Loss: 0.9696
Epoch   0 Batch   95/269 - Train Accuracy: 0.4805, Validation Accuracy: 0.4941, Loss: 0.9464
Epoch   0 Batch   96/269 - Train Accuracy: 0.4610, Validation Accuracy: 0.4815, Loss: 0.9304
Epoch   0 Batch   97/269 - Train Accuracy: 0.4430, Validation Accuracy: 0.4759, Loss: 0.9308
Epoch   0 Batch   98/269 - Train Accuracy: 0.4908, Validation Accuracy: 0.5051, Loss: 0.9151
Epoch   0 Batch   99/269 - Train Accuracy: 0.4689, Validation Accuracy: 0.5119, Loss: 0.9563
Epoch   0 Batch  100/269 - Train Accuracy: 0.5004, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.5290, Validation Accuracy: 0.5197, Loss: 0.6848
Epoch   0 Batch  180/269 - Train Accuracy: 0.5214, Validation Accuracy: 0.5298, Loss: 0.6776
Epoch   0 Batch  181/269 - Train Accuracy: 0.5133, Validation Accuracy: 0.5273, Loss: 0.6804
Epoch   0 Batch  182/269 - Train Accuracy: 0.5268, Validation Accuracy: 0.5423, Loss: 0.6834
Epoch   0 Batch  183/269 - Train Accuracy: 0.5849, Validation Accuracy: 0.5414, Loss: 0.5931
Epoch   0 Batch  184/269 - Train Accuracy: 0.5115, Validation Accuracy: 0.5407, Loss: 0.7042
Epoch   0 Batch  185/269 - Train Accuracy: 0.5486, Validation Accuracy: 0.5468, Loss: 0.6717
Epoch   0 Batch  186/269 - Train Accuracy: 0.4924, Validation Accuracy: 0.5368, Loss: 0.6944
Epoch   0 Batch  187/269 - Train Accuracy: 0.5276, Validation Accuracy: 0.5368, Loss: 0.6647
Epoch   0 Batch  188/269 - Train Accuracy: 0.5179, Validation Accuracy: 0.5314, Loss: 0.6580
Epoch   0 Batch  189/269 - Train Accuracy: 0.5386, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.5716, Validation Accuracy: 0.5810, Loss: 0.6030
Epoch   1 Batch    2/269 - Train Accuracy: 0.5707, Validation Accuracy: 0.5867, Loss: 0.5873
Epoch   1 Batch    3/269 - Train Accuracy: 0.5760, Validation Accuracy: 0.5876, Loss: 0.5978
Epoch   1 Batch    4/269 - Train Accuracy: 0.5550, Validation Accuracy: 0.5882, Loss: 0.6146
Epoch   1 Batch    5/269 - Train Accuracy: 0.5641, Validation Accuracy: 0.5721, Loss: 0.6030
Epoch   1 Batch    6/269 - Train Accuracy: 0.5926, Validation Accuracy: 0.5770, Loss: 0.5638
Epoch   1 Batch    7/269 - Train Accuracy: 0.5993, Validation Accuracy: 0.5793, Loss: 0.5616
Epoch   1 Batch    8/269 - Train Accuracy: 0.5786, Validation Accuracy: 0.5922, Loss: 0.6006
Epoch   1 Batch    9/269 - Train Accuracy: 0.5936, Validation Accuracy: 0.6059, Loss: 0.5892
Epoch   1 Batch   10/269 - Train Accuracy: 0.6016, Validation Accuracy: 0.6061, Loss: 0.5887
Epoch   1 Batch   11/269 - Train Accuracy: 0.6064, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.6208, Validation Accuracy: 0.6515, Loss: 0.5011
Epoch   1 Batch   91/269 - Train Accuracy: 0.6453, Validation Accuracy: 0.6412, Loss: 0.4595
Epoch   1 Batch   92/269 - Train Accuracy: 0.6355, Validation Accuracy: 0.6415, Loss: 0.4699
Epoch   1 Batch   93/269 - Train Accuracy: 0.6396, Validation Accuracy: 0.6430, Loss: 0.4494
Epoch   1 Batch   94/269 - Train Accuracy: 0.6446, Validation Accuracy: 0.6417, Loss: 0.4792
Epoch   1 Batch   95/269 - Train Accuracy: 0.6430, Validation Accuracy: 0.6371, Loss: 0.4701
Epoch   1 Batch   96/269 - Train Accuracy: 0.6514, Validation Accuracy: 0.6502, Loss: 0.4735
Epoch   1 Batch   97/269 - Train Accuracy: 0.6407, Validation Accuracy: 0.6625, Loss: 0.4612
Epoch   1 Batch   98/269 - Train Accuracy: 0.6523, Validation Accuracy: 0.6606, Loss: 0.4637
Epoch   1 Batch   99/269 - Train Accuracy: 0.6346, Validation Accuracy: 0.6536, Loss: 0.4795
Epoch   1 Batch  100/269 - Train Accuracy: 0.6606, Validation Accuracy

...

Epoch  10 Batch  179/269 - Train Accuracy: 0.9902, Validation Accuracy: 0.9700, Loss: 0.0068
Epoch  10 Batch  180/269 - Train Accuracy: 0.9919, Validation Accuracy: 0.9690, Loss: 0.0080
Epoch  10 Batch  181/269 - Train Accuracy: 0.9852, Validation Accuracy: 0.9703, Loss: 0.0122
Epoch  10 Batch  182/269 - Train Accuracy: 0.9862, Validation Accuracy: 0.9681, Loss: 0.0097
Epoch  10 Batch  183/269 - Train Accuracy: 0.9890, Validation Accuracy: 0.9680, Loss: 0.0065
Epoch  10 Batch  184/269 - Train Accuracy: 0.9794, Validation Accuracy: 0.9668, Loss: 0.0097
Epoch  10 Batch  185/269 - Train Accuracy: 0.9932, Validation Accuracy: 0.9663, Loss: 0.0072
Epoch  10 Batch  186/269 - Train Accuracy: 0.9857, Validation Accuracy: 0.9686, Loss: 0.0097
Epoch  10 Batch  187/269 - Train Accuracy: 0.9892, Validation Accuracy: 0.9723, Loss: 0.0091
Epoch  10 Batch  188/269 - Train Accuracy: 0.9908, Validation Accuracy: 0.9727, Loss: 0.0090
Epoch  10 Batch  189/269 - Train Accuracy: 0.9921, Validation Accuracy

Epoch  11 Batch    1/269 - Train Accuracy: 0.9871, Validation Accuracy: 0.9754, Loss: 0.0097
Epoch  11 Batch    2/269 - Train Accuracy: 0.9874, Validation Accuracy: 0.9782, Loss: 0.0093
Epoch  11 Batch    3/269 - Train Accuracy: 0.9908, Validation Accuracy: 0.9792, Loss: 0.0066
Epoch  11 Batch    4/269 - Train Accuracy: 0.9847, Validation Accuracy: 0.9727, Loss: 0.0093
Epoch  11 Batch    5/269 - Train Accuracy: 0.9895, Validation Accuracy: 0.9766, Loss: 0.0083
Epoch  11 Batch    6/269 - Train Accuracy: 0.9914, Validation Accuracy: 0.9781, Loss: 0.0081
Epoch  11 Batch    7/269 - Train Accuracy: 0.9874, Validation Accuracy: 0.9775, Loss: 0.0090
Epoch  11 Batch    8/269 - Train Accuracy: 0.9892, Validation Accuracy: 0.9803, Loss: 0.0105
Epoch  11 Batch    9/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9813, Loss: 0.0083
Epoch  11 Batch   10/269 - Train Accuracy: 0.9883, Validation Accuracy: 0.9784, Loss: 0.0086
Epoch  11 Batch   11/269 - Train Accuracy: 0.9888, Validation Accuracy

Epoch  11 Batch   90/269 - Train Accuracy: 0.9856, Validation Accuracy: 0.9743, Loss: 0.0097
Epoch  11 Batch   91/269 - Train Accuracy: 0.9905, Validation Accuracy: 0.9746, Loss: 0.0077
Epoch  11 Batch   92/269 - Train Accuracy: 0.9857, Validation Accuracy: 0.9724, Loss: 0.0087
Epoch  11 Batch   93/269 - Train Accuracy: 0.9876, Validation Accuracy: 0.9736, Loss: 0.0084
Epoch  11 Batch   94/269 - Train Accuracy: 0.9883, Validation Accuracy: 0.9743, Loss: 0.0095
Epoch  11 Batch   95/269 - Train Accuracy: 0.9917, Validation Accuracy: 0.9757, Loss: 0.0067
Epoch  11 Batch   96/269 - Train Accuracy: 0.9803, Validation Accuracy: 0.9760, Loss: 0.0106
Epoch  11 Batch   97/269 - Train Accuracy: 0.9850, Validation Accuracy: 0.9764, Loss: 0.0089
Epoch  11 Batch   98/269 - Train Accuracy: 0.9867, Validation Accuracy: 0.9751, Loss: 0.0096
Epoch  11 Batch   99/269 - Train Accuracy: 0.9932, Validation Accuracy: 0.9767, Loss: 0.0093
Epoch  11 Batch  100/269 - Train Accuracy: 0.9947, Validation Accuracy

Epoch  11 Batch  179/269 - Train Accuracy: 0.9911, Validation Accuracy: 0.9793, Loss: 0.0076
Epoch  11 Batch  180/269 - Train Accuracy: 0.9906, Validation Accuracy: 0.9776, Loss: 0.0072
Epoch  11 Batch  181/269 - Train Accuracy: 0.9891, Validation Accuracy: 0.9747, Loss: 0.0113
Epoch  11 Batch  182/269 - Train Accuracy: 0.9900, Validation Accuracy: 0.9767, Loss: 0.0080
Epoch  11 Batch  183/269 - Train Accuracy: 0.9946, Validation Accuracy: 0.9774, Loss: 0.0062
Epoch  11 Batch  184/269 - Train Accuracy: 0.9839, Validation Accuracy: 0.9745, Loss: 0.0097
Epoch  11 Batch  185/269 - Train Accuracy: 0.9906, Validation Accuracy: 0.9737, Loss: 0.0101
Epoch  11 Batch  186/269 - Train Accuracy: 0.9843, Validation Accuracy: 0.9741, Loss: 0.0076
Epoch  11 Batch  187/269 - Train Accuracy: 0.9857, Validation Accuracy: 0.9727, Loss: 0.0086
Epoch  11 Batch  188/269 - Train Accuracy: 0.9904, Validation Accuracy: 0.9717, Loss: 0.0067
Epoch  11 Batch  189/269 - Train Accuracy: 0.9911, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [227, 63, 45, 74, 65, 154, 90]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [22, 23, 45, 344, 243, 319, 39, 1]
  French Words: il a vu un camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.